In [1]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import string
import nltk
nltk.download('stopwords')
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import preprocessing
# from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier

[nltk_data] Error loading stopwords: <urlopen error [Errno -2] Name or
[nltk_data]     service not known>


In [2]:
#read the data set
data = pd.read_csv("./Data/phrases.csv")
# data = pd.read_csv("./Data/phrases.csv")
# data=pd.DataFrame(data)
# print type(data)
# print data.shape
data=data.dropna()
print data['Emotion'].unique()
# print data.shape

['joy' 'fear' 'anger' 'sadness' 'disgust' 'shame' 'guilt']


## Preprocess the data set via cleaning, tokenisation and lemmatization

In [3]:
# print data
import re

def clean_dataset(data):
    translator = string.maketrans('', '')
    for index,row in data.iterrows():
#         print index,row['Phrase']
        row['Phrase'] = row['Phrase'].replace('[','')
        row['Phrase'] = row['Phrase'].replace(']','')
        row['Phrase'] = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','URL', row['Phrase'])
        row['Phrase'] = re.sub('@[^\s]+','USER', row['Phrase'])
        row['Phrase'] = row['Phrase'].lower().replace("ё", "е")
        row['Phrase'] = re.sub('[^a-zA-Zа-яА-Я1-9]+', ' ', row['Phrase'])
        row['Phrase'] = re.sub(' +',' ', row['Phrase'])
        row['Phrase'] = row['Phrase'].strip()
        row['Phrase'] = row['Phrase'].translate(translator,string.punctuation)
#         print row['Phrase']
    return data
data = clean_dataset(data)
print data.head()

   Emotion                                             Phrase
0      joy  on days when i feel closing to my partner and ...
1     fear  every time i imagine that someone i love or i ...
2    anger  when i had been obviously unjustly treated and...
3  sadness  when i think about the short time that we live...
4  disgust  at a gathering i found myself involuntarily si...
